<a href="https://colab.research.google.com/github/Transega/Satelite-Image-classification/blob/main/Image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installs geemap package
import subprocess
 
try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
 
# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap
 
# Authenticates and initializes Earth Engine
import ee
 
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=MdiQh8VcQ_Ix4MSBHSAmXLYtGYRIm1hqOPeVRkZRRHI&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4gUqtxMlTgfvDSRD7jN-I3c5Vhmw3-44pVr48RkC5c8qoouguNyY8

Successfully saved authorization token.


In [ ]:
!pip install cartopy scipy
!pip install geemap

In [ ]:
!pip -q install rasterio

In [ ]:
pip install geopandas 

Import libraries

In [ ]:
import ee
import geemap
import geemap.eefolium
# import rasterio
# from rasterio import plot
# %matplotlib inline

Defination of area of study

In [ ]:
tharaka = './drive/MyDrive/omdena/tharaka_shapefile/tharaka_nithi.shp' #link to shapefile https://drive.google.com/drive/folders/1wJtqU6VDFlDMUaNgSuIGqIOWRZsrV4m8?usp=sharing
#countries_shp = '../data/countries.shp'
Tharaka_cnty = geemap.shp_to_ee(tharaka)
geometry = Tharaka_cnty

In [ ]:
# Counties Affected Area of interest
Counties_affected = ee.FeatureCollection('users/collinsasegaca/Affected_counties')
Map_a= geemap.Map()
Map_a.addLayer(Counties_affected)
Map_a.centerObject(Counties_affected, 6)
#Map_a

Defining the dates

In [ ]:
# date before attack
fdate = '2019-08-1' #('2017-01-1' ,'2017-12-31')
ldate = '2019-12-25'

#date after attack
after_fdate = '2020-1-1'
after_ldate = '2020-5-31'


Code to search and plot the images 

In [ ]:
Map= geemap.Map()
collection = ee.ImageCollection("COPERNICUS/S2") \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)) \
    .filterBounds(geometry) \

before_collection = collection.filterDate(fdate,ldate)# before attack
after_collection = collection.filterDate(after_fdate,after_ldate)# after attack

medianpixels_before = before_collection.median()
medianpixels_after = after_collection.median()
medianpixelsclipped_before = medianpixels_before.clip(geometry).divide(10000)
medianpixelsclipped_after = medianpixels_after.clip(geometry).divide(10000) 


vis_params = {
    'min': 0,
    'max': 1,
    'bands': ['B12', 'B11', 'B4'], 'gamma': 1.5
}


Map.centerObject(geometry, 8)
Map.addLayer(medianpixelsclipped_before, vis_params, 'Sentinel_2 mosaic_before')
Map.addLayer(medianpixelsclipped_after, vis_params, 'Sentinel_2 mosaic_after')
Map.addLayer(geometry)
Map.addLayerControl()


Image Search for the remaining counties

Checking image properties

In [ ]:

medianpixelsclipped_before.get('CLOUD_COVER').getInfo()

In [ ]:
#tutorial on how to clasify
geemap.show_youtube('qWaEfgWi21o') #lwtgzrHrXj8

Output(layout=Layout(width='815px'))

Will be using Land use land cover 2016 as the training data (data was provided by Regional Center for mapping of resources for development) link to the data https://drive.google.com/drive/folders/1UmqRHXOPrmVhzq0lj55F87VVq5VHySVi?usp=sharing

In [ ]:
#since the data covers the entire country I clipped to fit tharaka nithi county before I try on other counties.
S2LULC_2016 = ee.Image('users/collinsasegaca/Kenya_Sentinel2_LULC2016').select('b1').clip(geometry.geometry())

In [ ]:
vis = {
    'min': 0,
    'max': 1,
    'bands': ['b1'], 'gamma': 1.5
}

In [ ]:
Map3 = geemap.Map()
Map3.centerObject(geometry, 8)
Map3.addLayer(S2LULC_2016, {'min': 1, 'max': 8,'palette': ['1c5f2c', '0000FF','#3c1f46','#3a1e45', '#3b1f45','6a2325']}, 'S2 ke')
# M = geemap.Map()
# M.addLayer(result,
#              {min: 0, max: 10, 'palette':VizPalette  },
#              'IGBP classification');

In [ ]:
Map3

Generation training points by using the LULC and obtaining random points

In [ ]:
# Make the training dataset.
points = S2LULC_2016.sample(**{
    'region': geometry.geometry(),
    'scale': 10,
    'numPixels': 3000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map3.addLayer(points.draw("blue"), {}, 'training', False);

In [ ]:
# Am unable to Visualize the GEE object that i uploaded into RGB but the points can be used for training
Map3


In [ ]:
print(points.size().getInfo())

2998


In [ ]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [38.031335722303815, -0.2067486453872509]}, 'id': '0', 'properties': {'b1': 4}}


In [ ]:
bands = ['B2', 'B3', 'B4', 'B8']


# This property of the table stores the land cover labels.
label = 'b1'

# Overlay the points on the imagery to get training for the before imge
training = medianpixelsclipped_before.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 10
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [ ]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B2': 0.1115, 'B3': 0.1195, 'B4': 0.1677, 'B8': 0.2188, 'b1': 4}}


In [ ]:
print(training.getInfo())

{'type': 'FeatureCollection', 'columns': {}, 'properties': {'band_order': ['B2', 'B3', 'B4', 'B8']}, 'features': [{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B2': 0.1115, 'B3': 0.1195, 'B4': 0.1677, 'B8': 0.2188, 'b1': 4}}, {'type': 'Feature', 'geometry': None, 'id': '1_0', 'properties': {'B2': 0.1003, 'B3': 0.104, 'B4': 0.1156, 'B8': 0.2476, 'b1': 1}}, {'type': 'Feature', 'geometry': None, 'id': '2_0', 'properties': {'B2': 0.067, 'B3': 0.0659, 'B4': 0.0374, 'B8': 0.2734, 'b1': 1}}, {'type': 'Feature', 'geometry': None, 'id': '3_0', 'properties': {'B2': 0.1013, 'B3': 0.1007, 'B4': 0.1404, 'B8': 0.2182, 'b1': 4}}, {'type': 'Feature', 'geometry': None, 'id': '4_0', 'properties': {'B2': 0.0859, 'B3': 0.0826, 'B4': 0.0882, 'B8': 0.2085, 'b1': 4}}, {'type': 'Feature', 'geometry': None, 'id': '5_0', 'properties': {'B2': 0.1055, 'B3': 0.1093, 'B4': 0.1358, 'B8': 0.2387, 'b1': 4}}, {'type': 'Feature', 'geometry': None, 'id': '6_0', 'properties': {'B2': 0.1077, 'B3': 0.11

In [ ]:
legends = geemap.builtin_legends
for legend in legends:
    print(legend)

NLCD
NWI
MODIS/051/MCD12Q1
GLOBCOVER
JAXA/PALSAR
MODIS/006/MCD12Q1
Oxford
AAFC/ACI
COPERNICUS/CORINE/V20/100m
COPERNICUS/Landcover/100m/Proba-V/Global
USDA/NASS/CDL


In [ ]:
legend_keys = ['Trees cover areas ', 'Shrubs cover areas ', 'Grassland ', 'Cropland ', 'Vegetation aquatic or regularly flooded','Lichen Mosses / Sparse vegetation ','Bare areas ','Built up areas','Snow and/or Ice ', 'Open water']
legend_colors = ['1c5f2c','ccb879','dfdfc2','ab6c28','6c9fb8','a3cc51','b3ac9f','ab0000','d1def8','466b9f' ]

In [ ]:
legend_dict = {
    '1 Trees cover areas': '1c5f2c',
    '2 Shrubs cover areas': 'ccb879',
    '3 Grassland': 'dfdfc2',
    '4 Cropland': 'ab6c28',
    '5 Vegetation aquatic or regularly flooded': '6c9fb8',
    '6 Lichen Mosses / Sparse vegetation ': 'a3cc51',
    '7 Bare areas': 'Built up areas',
    '8 Built up areas': 'ab0000',
    '9 Snow and/or Ice': 'd1def8',
    '10 Open water': '466b9f'
}

In [ ]:
# Classify the image with the same bands used for training.
result = medianpixelsclipped_before.select(bands).classify(trained)

# # Display the clusters with random colors.
Map= geemap.Map()

legend_dict = {
    '1 Trees cover areas': '1c5f2c',
    '2 Shrubs cover areas': 'ccb879',
    '3 Grassland': 'dfdfc2',
    '4 Cropland': 'ab6c28',
    '5 Vegetation aquatic or regularly flooded': '6c9fb8',
    '6 Lichen Mosses / Sparse vegetation ': 'a3cc51',
    '7 Bare areas': 'Built up areas',
    '8 Built up areas': 'ab0000',
    '9 Snow and/or Ice': 'd1def8',
    '10 Open water': '466b9f'
}
Map.centerObject(geometry,8)
Map.addLayer(result, {'min': 1, 'max': 10,'palette': ['1c5f2c','ccb879','dfdfc2','ab6c28','6c9fb8','a3cc51','b3ac9f','ab0000','d1def8','466b9f' ]}, 'classfied')
Map.add_legend(legend_dict=legend_dict, title="Tharaka LULC Classification")
Map.addLayerControl()
Map

The provided color is invalid. Using the default black color.
'#Built up areas' is not in web format. Need 3 or 6 hex digit.


In [ ]:
from geemap import cartoee

In [ ]:
geemap.ee_export_image_to_drive(result, description='landcover_tharaka', folder='omdena', scale=900)

Exporting landcover_tharaka ...


In [ ]:
legends = geemap.builtin_legends
for legend in legends:
    print(legend)


NLCD
NWI
MODIS/051/MCD12Q1
GLOBCOVER
JAXA/PALSAR
MODIS/006/MCD12Q1
Oxford
AAFC/ACI
COPERNICUS/CORINE/V20/100m
COPERNICUS/Landcover/100m/Proba-V/Global
USDA/NASS/CDL


In [ ]:
result.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 2147483647,
    'min': -2147483648,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'classification'}],
 'type': 'Image'}

trial on accuracy assesment 

In [ ]:
#Split Training and testing

bands = ['B2', 'B3', 'B4', 'B8']


# This property of the table stores the land cover labels.
label = 'b1'

# Overlay the points on the imagery to get training for the before imge
sample = medianpixelsclipped_before.select(bands).sampleRegions(**{
 'collection': points,
 'properties': [label],
  'scale': 10
 })

# Adds a column of deterministic pseudorandom numbers. 
sample = sample.randomColumn()

split = 0.7

training_dta = sample.filter(ee.Filter.lt('random', split))
validation_dta = sample.filter(ee.Filter.gte('random', split))

In [ ]:
training_dta.first().getInfo()

{'geometry': None,
 'id': '0_0',
 'properties': {'B2': 0.1115,
  'B3': 0.1195,
  'B4': 0.1677,
  'B8': 0.2188,
  'b1': 4,
  'random': 0.19464472010081202},
 'type': 'Feature'}

In [ ]:
validation_dta.first().getInfo()

{'geometry': None,
 'id': '1_0',
 'properties': {'B2': 0.1003,
  'B3': 0.104,
  'B4': 0.1156,
  'B8': 0.2476,
  'b1': 1,
  'random': 0.7563798285188889},
 'type': 'Feature'}

# Train the classifier (smile random forest)

In [ ]:
classifier = ee.Classifier.smileRandomForest(10).train(training_dta, label, bands)

Classify the image 

In [ ]:
# Classify the image with the same bands used for training.
result_1 = medianpixelsclipped_before.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map4 = geemap.Map()
Map4.addLayer(result_1.randomVisualizer(), {}, 'classfied')
Map4.addLayerControl()
Map4.centerObject(geometry, 8)
Map4

In [ ]:
#accuracy assesment 

train_accuracy = classifier.confusionMatrix()

In [ ]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 312, 0, 2, 11, 0, 0, 0, 0, 0, 0],
 [0, 2, 129, 0, 39, 0, 0, 0, 0, 0, 0],
 [0, 2, 1, 70, 9, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 1, 1496, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]

In [ ]:

train_accuracy.accuracy().getInfo()

0.9646102343376375

### The Kappa Coefficient is generated from a statistical test to evaluate the accuracy of a classification. Kappa essentially evaluates how well the classification performed as compared to just randomly assigning values, i.e. did the classification do better than random. The Kappa Coefficient can range from -1 t0 1. A value of 0 indicated that the classification is no better than a random classification. A negative number indicates the classification is significantly worse than random. A value close to 1 indicates that the classification is significantly better than random

In [ ]:

train_accuracy.kappa().getInfo()

0.9219784386903818

Producer's Accuracy is the map accuracy from the point of view of the map maker (the producer). This is how often are real features on the ground correctly shown on the classified map or the probability that a certain land cover of an area on the ground is classified as such. The Producer's Accuracy is complement of the Omission Error, Producer's Accuracy = 100%-Omission Error. It is also the number of reference sites classified accurately divided by the total number of reference sites for that class.

In [ ]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0.9650145772594753],
 [0.7953216374269005],
 [0.7948717948717948],
 [0.9953145917001339],
 [0],
 [0],
 [0],
 [0.6]]

**Validation dataset**

In [ ]:
validated = validation_dta.classify(classifier)

In [ ]:
validated.first().getInfo()

{'geometry': None,
 'id': '7_0',
 'properties': {'B2': 0.1053,
  'B3': 0.1105,
  'B4': 0.16,
  'B8': 0.2042,
  'b1': 4,
  'classification': 4,
  'random': 0.7740711446923665},
 'type': 'Feature'}

errorMatrix computes a 2D error matrix for a collection by comparing two columns of a collection: one containing the actual values, and one containing predicted values.

In [ ]:
test_accuracy = validated.errorMatrix('b1', 'classification')

In [ ]:
test_accuracy.accuracy().getInfo()